In [11]:
import pickle
from pathlib import Path
from smount_predictors import SeamountHelp
import plotly.express as px
from sklearn.cluster import HDBSCAN
from sklearn.pipeline import Pipeline

In [6]:
model = pickle.load(open('seamount_predictor.pkl', 'rb'))
points = SeamountHelp.readKMLbounds(Path('data/seamount_training_zone.kml'))
data = SeamountHelp.readAndFilterGRD(Path('data/vgg_swot.grd'), points[:2], points[2:]).to_dataframe().reset_index()
predictions = model.predict(data)

In [9]:
data['class'] = predictions

In [7]:
px.scatter(x=data['lon'], y=data['lat'], color=predictions).show()

In [13]:
clusterer = HDBSCAN()
clusterer.fit(data[['lon', 'lat', 'class']])
data['cluster'] = clusterer.labels_
px.scatter(data[data['cluster'] != 0], x='lon', y='lat', color='cluster').show()

In [19]:
class PipelinePredictor:
    def __init__(self, model, clusterer):
        self.model = model
        self.clusterer = clusterer

    def predict(self, data):
        predictions = self.model.predict(data)
        data['class'] = predictions
        self.clusterer.fit_predict(data[['lon', 'lat', 'class']])
        data['cluster'] = self.clusterer.labels_
        return data

In [20]:
full_pipeline = PipelinePredictor(model, clusterer)
pickle.dump(full_pipeline, open('seamount_predictor_full.pkl', 'wb'))

In [21]:
large_zone = [(40, 55), (-165, -130)]
test_zone = SeamountHelp.readAndFilterGRD(Path('data/vgg_swot.grd'), large_zone[0], large_zone[1]).to_dataframe().reset_index()
test_predictions = full_pipeline.predict(test_zone)
test_zone['class'] = test_predictions
px.scatter(x=test_zone['lon'], y=test_zone['lat'], color=test_predictions).show()

AttributeError: This 'GridSearchCV' has no attribute 'transform'